<a href="https://colab.research.google.com/github/maxysio/cp_ortools/blob/master/EmployeeScheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install --upgrade --user ortools

     |████████████████████████████████| 28.9MB 155kB/s 
     |████████████████████████████████| 1.3MB 51.1MB/s 


# Employee Scheduling

Plants have employees who work multiple shifts. It is required to schedule sufficient workers for each daily shift. Typically, the schedules will have constraints, such as "no employee should work two shifts in a row". Finding a schedule that satisfies all these criterias constraints can be challenging. 

In this example we create a schedule for a plant
- Total number of Employees 10
- Schedule Period is for 7 days
- Each day is divided into three 8 hour shifts
- Maximum number of employees in a shift in the plant cannot exceed 5
- An employee cannot have more than 1 shift in a day
- An employee should get at least 5 shifts in 7 days

In [0]:
from __future__ import print_function
from ortools.sat.python import cp_model

In [0]:
import pandas as pd
import numpy as np

In [0]:
num_employees = 20
num_shifts = 3
num_days = 7
employees_per_shift = 5

all_employees = range(num_employees)
all_shifts = range(num_shifts)
all_days = range(num_days)

In [0]:
model = cp_model.CpModel()

#### Decision Variable

- shift[e,d,s] = 1 when employee e works on day d in shift s

In [0]:
# Create the decision Variable
shifts = {}
for e in all_employees:
  for d in all_days:
    for s in all_shifts:
      shifts[(e, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (e, d, s))

In [0]:
# Constraint 1
# Maximum number of employees in a shift cannot exceed 8
for d in all_days:
  for s in all_shifts:
    model.Add(sum(shifts[(e, d, s)] for e in all_employees) <= employees_per_shift)

In [0]:
# Constraint 2
# An employee cannot have more than 1 shift in a day
for e in all_employees:
  for d in all_days:
    model.Add(sum(shifts[(e, d, s)] for s in all_shifts) <= 1)

In [0]:
min_shifts_per_employee = int(np.floor(((num_shifts * num_days) / num_employees) * employees_per_shift))

In [0]:
min_shifts_per_employee = (num_shifts * num_days) // num_employees

In [105]:
min_shifts_per_employee

5

In [0]:
max_shifts_per_employee = min_shifts_per_employee + 1

In [0]:
for e in all_employees:
  num_shifts_worked = sum(
      shifts[(e, d, s)] for d in all_days for s in all_shifts)
  
  model.Add(min_shifts_per_employee <= num_shifts_worked)
  model.Add(num_shifts_worked <= max_shifts_per_employee)

In [0]:
class EmployeePartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_employees, num_days, num_shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_employees = num_employees
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solutions = set(sols)
        self._solution_count = 0

    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print('Solution %i' % self._solution_count)
            for d in range(self._num_days):
                print('Day %i' % d)
                for n in range(self._num_employees):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.Value(self._shifts[(n, d, s)]):
                            is_working = True
                            print('  Employee %i works shift %i' % (n, s))
                    if not is_working:
                        print('  Employee {} does not work'.format(n))
            print()

            shift_cols = []
            for d in range(self._num_days):
              for s in range(self._num_shifts):
                shift_cols.append('Day' + str(d+1) + '-Shift' + str(s+1))
            
            emp_rows = []
            for n in range(self._num_employees):
              emp_rows.append('Emp' + str(n+1))

            df_schedule = pd.DataFrame(columns=shift_cols, index=emp_rows, data=)

        self._solution_count += 1

    def solution_count(self):
        return self._solution_count

In [111]:
# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Display the first five solutions.
a_few_solutions = range(2)
solution_printer = EmployeePartialSolutionPrinter(shifts, num_employees, num_days, num_shifts, a_few_solutions)
solver.SearchForAllSolutions(model, solution_printer)

# Statistics.
print()
print('Statistics')
print('  - conflicts       : %i' % solver.NumConflicts())
print('  - branches        : %i' % solver.NumBranches())
print('  - wall time       : %f s' % solver.WallTime())
print('  - solutions found : %i' % solution_printer.solution_count())

Solution 0
Day 0
  Employee 0 does not work
  Employee 1 does not work
  Employee 2 does not work
  Employee 3 works shift 1
  Employee 4 works shift 1
  Employee 5 works shift 1
  Employee 6 works shift 2
  Employee 7 works shift 0
  Employee 8 works shift 1
  Employee 9 works shift 1
  Employee 10 works shift 0
  Employee 11 works shift 0
  Employee 12 works shift 0
  Employee 13 works shift 0
  Employee 14 does not work
  Employee 15 works shift 2
  Employee 16 works shift 2
  Employee 17 works shift 2
  Employee 18 does not work
  Employee 19 works shift 2
Day 1
  Employee 0 does not work
  Employee 1 works shift 2
  Employee 2 does not work
  Employee 3 works shift 0
  Employee 4 does not work
  Employee 5 works shift 2
  Employee 6 does not work
  Employee 7 works shift 2
  Employee 8 works shift 0
  Employee 9 works shift 0
  Employee 10 works shift 2
  Employee 11 works shift 2
  Employee 12 works shift 0
  Employee 13 does not work
  Employee 14 works shift 1
  Employee 15 wor